In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torchmetrics.functional as F_tm
import torchvision 
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset, Subset
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
import numpy as np
import random
import time
import copy
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"Usando GPU: {torch.cuda.get_device_name(0)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data_path = '/Users/luanr/pycharm/FedAlzheimer/FedAlzheimer/data/ADNI 4'

In [ ]:
class RandomNoise(object):
    def __init__(self, p=0.5, mean=0., std=0.1): self.p,self.mean,self.std=p,mean,std
    def __call__(self, tensor):
        if torch.rand(1).item() < self.p: return tensor + torch.randn(tensor.size(), device=tensor.device) * self.std + self.mean
        return tensor
    def __repr__(self): return f"{self.__class__.__name__}(p={self.p}, mean={self.mean}, std={self.std})"

class CustomDataset(Dataset):
    def __init__(self, subset, transform=None): self.subset, self.transform = subset, transform
    def __getitem__(self, index):
        image, label = self.subset[index]
        if self.transform: image = self.transform(image)
        return image, label
    def __len__(self): return len(self.subset)

class ResNet50_client_side(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        for name, param in resnet.named_parameters():
            if name.startswith(("conv1", "bn1")): param.requires_grad = False
        self.features = nn.Sequential(resnet.conv1, resnet.bn1, resnet.relu, resnet.maxpool, resnet.layer1, resnet.layer2)
    def forward(self, x): return self.features(x)

class ResNet50_server_side(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()
        resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.features = nn.Sequential(resnet.layer3, resnet.layer4)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        num_ftrs = resnet.fc.in_features
        self.classifier = nn.Sequential(nn.Dropout(p=0.5), nn.Linear(num_ftrs, num_classes))
    def forward(self, x): x=self.features(x); x=self.pool(x); x=torch.flatten(x,1); return self.classifier(x)

class CombinedModel(nn.Module):
    def __init__(self, client_model, server_model): super().__init__();self.client_model=client_model;self.server_model=server_model
    def forward(self, x): return self.server_model(self.client_model(x))

class EarlyStopping:
    def __init__(self,patience=10,verbose=True,delta=0,save_path='chkp.pt'): self.patience,self.verbose,self.delta,self.save_path=patience,verbose,delta,save_path; self.counter,self.best_score,self.early_stop,self.val_loss_min=0,None,False,np.Inf
    def __call__(self,val_loss,models):
        score=-val_loss; net_client, net_server = models
        if self.best_score is None: self.best_score=score; self.save_checkpoint(val_loss, net_client, net_server)
        elif score<self.best_score+self.delta:
            self.counter+=1
            if self.verbose:print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter>=self.patience: self.early_stop=True
        else: self.best_score=score; self.save_checkpoint(val_loss, net_client, net_server); self.counter=0
    def save_checkpoint(self,val_loss,net_client,net_server):
        if self.verbose:print(f'Validation loss decreased ({self.val_loss_min:.6f}-->{val_loss:.6f}). Saving model to {self.save_path}')
        torch.save({'net_glob_client_state_dict':net_client.state_dict(),'net_glob_server_state_dict':net_server.state_dict()}, self.save_path); self.val_loss_min=val_loss

def FedAvg(w_locals):
    if not w_locals:
        return {}

    w_avg = copy.deepcopy(w_locals[0])

    for k in w_avg.keys():
        if not w_avg[k].dtype.is_floating_point:
            continue

        stacked_tensors = torch.stack([w[k].float() for w in w_locals], dim=0)
        w_avg[k] = stacked_tensors.mean(dim=0)

    return w_avg

def calculate_metrics(fx, y, num_classes):
    preds=fx.argmax(dim=1)
    acc=F_tm.accuracy(preds,y,task='multiclass',num_classes=num_classes)
    prec=F_tm.precision(preds,y,average='macro',task='multiclass',num_classes=num_classes)
    rec=F_tm.recall(preds,y,average='macro',task='multiclass',num_classes=num_classes)
    f1=F_tm.f1_score(preds,y,average='macro',task='multiclass',num_classes=num_classes)
    try: auc=F_tm.auroc(F.softmax(fx,dim=1),y,task='multiclass',num_classes=num_classes)
    except ValueError: auc=torch.tensor(0.0, device=fx.device)
    return acc,prec,rec,f1,auc

def train_server(fx_client, y, net_server, optimizer_server, criterion, num_classes):
    net_server.train();optimizer_server.zero_grad()
    fx_server=net_server(fx_client);loss=criterion(fx_server,y)
    metrics=calculate_metrics(fx_server,y,num_classes)
    loss.backward();dfx_client=fx_client.grad.clone().detach();optimizer_server.step()
    return dfx_client,loss.item(),metrics

def evaluate_server(fx_client, y, net_server, criterion, num_classes):
    net_server.eval()
    with torch.no_grad():
        fx_server=net_server(fx_client);loss=criterion(fx_server,y)
        metrics=calculate_metrics(fx_server,y,num_classes)
    return loss.item(),metrics


class Client:
    def __init__(self, idx, lr, device, idxs, idxs_test, net_server_initial_weights, weight_decay):
        self.idx,self.device,self.lr,self.local_ep=idx,device,lr,1
        self.train_dataset=CustomDataset(Subset(full_dataset,list(idxs)),transform=train_transform)
        self.test_dataset=CustomDataset(Subset(full_dataset,list(idxs_test)),transform=test_transform)
        self.ldr_train=DataLoader(self.train_dataset,batch_size=batch_size,shuffle=True,num_workers=2,pin_memory=True)
        self.ldr_test=DataLoader(self.test_dataset,batch_size=batch_size,shuffle=False,num_workers=2,pin_memory=True)
        self.net_server_local_copy=ResNet50_server_side(num_classes=num_classes).to(self.device)
        self.net_server_local_copy.load_state_dict(net_server_initial_weights)
        self.optimizer_server_local=torch.optim.Adam(self.net_server_local_copy.parameters(),lr=self.lr,weight_decay=weight_decay)
        self.mu = mu

    def train(self, net_client):
        net_client.train()
        net_client.to(self.device)

        global_weights = copy.deepcopy(net_client.state_dict())

        optimizer_client = torch.optim.Adam(net_client.parameters(), lr=self.lr)

        batch_losses, batch_metrics = [], []
        for images, labels in self.ldr_train:
            images, labels = images.to(self.device), labels.to(self.device)
            optimizer_client.zero_grad()

            fx = net_client(images)
            client_fx = fx.clone().detach().requires_grad_(True)

            dfx, loss, metrics = train_server(client_fx, labels, self.net_server_local_copy, self.optimizer_server_local, criterion, num_classes)

            fx.backward(dfx)

            proximal_term = 0.0
            for name, param in net_client.named_parameters():
                if param.requires_grad:
                    proximal_term += torch.sum(torch.pow(param - global_weights[name].to(self.device), 2))

            prox_loss = (self.mu / 2) * proximal_term
            prox_loss.backward()
            # -------------------------

            optimizer_client.step()

            batch_losses.append(loss)
            batch_metrics.append([m.item() for m in metrics])

        avg_loss = sum(batch_losses) / len(batch_losses) if batch_losses else 0
        avg_metrics = [sum(col) / len(col) for col in zip(*batch_metrics)] if batch_metrics else [0]*5

        return net_client.state_dict(), self.net_server_local_copy.state_dict(), avg_loss, avg_metrics

    def evaluate(self, net_client, net_server):
        net_client.eval();net_server.eval();net_client.to(self.device);net_server.to(self.device)
        batch_losses,batch_metrics=[],[]
        with torch.no_grad():
            for images,labels in self.ldr_test:
                images,labels=images.to(self.device),labels.to(self.device)
                fx=net_client(images)
                loss,metrics=evaluate_server(fx,labels,net_server,criterion,num_classes)
                batch_losses.append(loss);batch_metrics.append([m.item() for m in metrics])
        avg_loss=sum(batch_losses)/len(batch_losses) if batch_losses else 0
        avg_metrics=[sum(col)/len(col) for col in zip(*batch_metrics)] if batch_metrics else [0]*5
        return avg_loss,avg_metrics

def evaluate_accuracy(net,loader,device,return_conf_matrix=False,num_classes=3):
    net.eval();all_preds,all_labels,all_outputs=[],[],[]
    with torch.no_grad():
        for images,labels in loader:
            images,labels=images.to(device),labels.to(device);outputs=net(images);_,predicted=torch.max(outputs,1);all_outputs.append(outputs.cpu());all_preds.extend(predicted.cpu().numpy());all_labels.extend(labels.cpu().numpy())
    all_outputs=torch.cat(all_outputs,dim=0);all_preds=torch.tensor(all_preds);all_labels=torch.tensor(all_labels);
    accuracy=F_tm.accuracy(all_preds,all_labels,task='multiclass',num_classes=num_classes).item();precision=F_tm.precision(all_preds,all_labels,average='macro',task='multiclass',num_classes=num_classes).item();recall=F_tm.recall(all_preds,all_labels,average='macro',task='multiclass',num_classes=num_classes).item();f1=F_tm.f1_score(all_preds,all_labels,average='macro',task='multiclass',num_classes=num_classes).item();auc=F_tm.auroc(F.softmax(all_outputs,dim=1),all_labels,task="multiclass",num_classes=num_classes).item();
    conf_matrix=confusion_matrix(all_labels.cpu().numpy(),all_preds.cpu().numpy())
    if return_conf_matrix:return accuracy,precision,recall,f1,auc,conf_matrix
    return accuracy,precision,recall,f1,auc

def dataset_iid(indices, num_users):
    indices = np.random.permutation(indices); splits = np.array_split(indices, num_users); return {i: set(splits[i]) for i in range(num_users)}


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.RandomAffine(degrees=10, translate=(0.1,0.1), scale=(0.85,1.15)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    RandomNoise(p=0.2, mean=0., std=0.08),
    transforms.Normalize([0.5]*3, [0.5]*3),
])
test_transform=transforms.Compose([transforms.Resize((224,224)),transforms.Grayscale(num_output_channels=3),transforms.ToTensor(),transforms.Normalize([0.5]*3,[0.5]*3)])
full_dataset=datasets.ImageFolder(root=data_path)
labels_all=[lbl for _,lbl in full_dataset.samples]
train_idx,test_idx=train_test_split(np.arange(len(full_dataset)),test_size=0.2,stratify=labels_all,random_state=SEED)
main_train_dataset=CustomDataset(Subset(full_dataset,train_idx),transform=train_transform)
main_test_dataset = CustomDataset(Subset(full_dataset, test_idx), transform=test_transform)

In [ ]:
client_counts = [3, 5, 7, 9]
num_classes = 3
batch_size = 32

for num_users in client_counts:
    print(f"\n{'='*80}\nINICIANDO TREINAMENTO PARA {num_users} CLIENTES\n{'='*80}\n")

    epochs,frac,lr,weight_decay=80,1.0,1e-4,1e-4
    mu = 0.01
    net_glob_client=ResNet50_client_side().to(device);net_glob_server=ResNet50_server_side(num_classes=num_classes).to(device)
    w_glob_client=net_glob_client.state_dict();w_glob_server=net_glob_server.state_dict()

    # Reinicializa listas de histórico de métricas
    history={'train_loss':[],'train_acc':[],'test_loss':[],'test_acc':[],'train_recall':[],'test_recall':[],'train_auc':[],'test_auc':[]}

    dict_users=dataset_iid(train_idx,num_users);dict_users_test=dataset_iid(test_idx,num_users)
    labels = [label for _, label in main_train_dataset]
    class_weights=compute_class_weight('balanced',classes=np.unique(labels),y=labels)
    criterion=nn.CrossEntropyLoss(weight=torch.tensor(class_weights,dtype=torch.float).to(device))

    checkpoint_path=f'best_model_{num_users}_clients.pt'
    early_stopping=EarlyStopping(patience=10,verbose=True,save_path=checkpoint_path)

    # --- LOOP DE TREINAMENTO (ESTRUTURA REATORADA) ---
    for iter_epoch in range(epochs):
        start_time=time.time()
        idxs_users=np.random.choice(range(num_users),max(int(frac*num_users),1),replace=False)

        w_locals_client,w_locals_server,round_train_loss,round_train_metrics=[],[],[],[]

        # ETAPA 1: TREINAMENTO LOCAL DOS CLIENTES
        for idx in idxs_users:
            local=Client(idx,lr,device,dict_users[idx],dict_users_test[idx],w_glob_server,weight_decay)
            w_c,w_s,t_loss,t_metrics=local.train(net_client=copy.deepcopy(net_glob_client))
            w_locals_client.append(w_c);w_locals_server.append(w_s);round_train_loss.append(t_loss);round_train_metrics.append(t_metrics)

        # ETAPA 2: AGREGAÇÃO NO SERVIDOR (FedAvg)
        w_glob_client = FedAvg(w_locals_client)
        w_glob_server = FedAvg(w_locals_server)
        net_glob_client.load_state_dict(w_glob_client)
        net_glob_server.load_state_dict(w_glob_server)

        # ETAPA 3: AVALIAÇÃO GLOBAL E COLETA DE MÉTRICAS
        round_test_loss,round_test_metrics=[],[]
        for idx in idxs_users:
            local=Client(idx,lr,device,dict_users[idx],dict_users_test[idx],w_glob_server,weight_decay)
            test_loss,test_metrics=local.evaluate(net_glob_client,net_glob_server)
            round_test_loss.append(test_loss);round_test_metrics.append(test_metrics)

        # Calcula médias da rodada e salva no histórico
        history['train_loss'].append(sum(round_train_loss)/len(round_train_loss))
        avg_train_metrics=[sum(col)/len(col) for col in zip(*round_train_metrics)]
        history['train_acc'].append(avg_train_metrics[0]);history['train_recall'].append(avg_train_metrics[2]);history['train_auc'].append(avg_train_metrics[4])

        history['test_loss'].append(sum(round_test_loss)/len(round_test_loss))
        avg_test_metrics=[sum(col)/len(col) for col in zip(*round_test_metrics)]
        history['test_acc'].append(avg_test_metrics[0]);history['test_recall'].append(avg_test_metrics[2]);history['test_auc'].append(avg_test_metrics[4])

        print(f"Epoch {iter_epoch+1}/{epochs} | Train Loss:{history['train_loss'][-1]:.4f} Acc:{history['train_acc'][-1]:.4f} | Test Loss:{history['test_loss'][-1]:.4f} Acc:{history['test_acc'][-1]:.4f} | Time:{time.time()-start_time:.2f}s")

        early_stopping(history['test_loss'][-1],(net_glob_client,net_glob_server))
        if early_stopping.early_stop:print("Early stopping triggered.");break

    print(f"\nTreinamento para {num_users} clientes concluído!")

    # --- AVALIAÇÃO E PLOTAGEM FINAL ---
    print("\nGerando gráficos e avaliação final...")
    if history['train_acc']:
        epochs_range=range(1,len(history['train_acc'])+1)
        plt.style.use('seaborn-v0_8-whitegrid')
        fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,7));fig.suptitle(f'Acurácia e Perda ({num_users} Clientes)',fontsize=20)
        ax1.plot(epochs_range,history['train_acc'],'o-',label='Acurácia de Treino');ax1.plot(epochs_range,history['test_acc'],'s-',label='Acurácia de Validação');ax1.set_title('Acurácia vs. Época');ax1.set_xlabel('Época');ax1.set_ylabel('Acurácia');ax1.legend()
        ax2.plot(epochs_range,history['train_loss'],'o-',label='Perda de Treino');ax2.plot(epochs_range,history['test_loss'],'s-',label='Perda de Validação');ax2.set_title('Perda vs. Época');ax2.set_xlabel('Época');ax2.set_ylabel('Perda');ax2.legend()
        plt.show()
        fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,7));fig.suptitle(f'Recall e AUC ({num_users} Clientes)',fontsize=20)
        ax1.plot(epochs_range,history['train_recall'],'o-',label='Recall de Treino');ax1.plot(epochs_range,history['test_recall'],'s-',label='Recall de Validação');ax1.set_title('Recall vs. Época');ax1.set_xlabel('Época');ax1.set_ylabel('Recall');ax1.legend()
        ax2.plot(epochs_range,history['train_auc'],'o-',label='AUC de Treino');ax2.plot(epochs_range,history['test_auc'],'s-',label='AUC de Validação');ax2.set_title('AUC vs. Época');ax2.set_xlabel('Época');ax2.set_ylabel('AUC');ax2.legend()
        plt.show()

    try:
        checkpoint=torch.load(checkpoint_path);net_glob_client.load_state_dict(checkpoint['net_glob_client_state_dict']);net_glob_server.load_state_dict(checkpoint['net_glob_server_state_dict'])
        final_model=CombinedModel(net_glob_client,net_glob_server).to(device)
        accuracy,precision,recall,f1,auc,conf_matrix=evaluate_accuracy(final_model,DataLoader(main_test_dataset,batch_size=batch_size,shuffle=False),device,return_conf_matrix=True,num_classes=num_classes)
        print(f"\n--- Métricas Finais ({num_users} Clientes) ---");print(f"  Acurácia:{accuracy:.4f}, Precisão:{precision:.4f}, Recall:{recall:.4f}, F1-Score:{f1:.4f}, AUC:{auc:.4f}")
        plt.figure(figsize=(10,8));sns.heatmap(conf_matrix,annot=True,fmt='d',cmap='Blues',xticklabels=range(num_classes),yticklabels=range(num_classes));plt.title(f'Matriz de Confusão ({num_users} Clientes)',fontsize=18);plt.xlabel('Rótulo Previsto');plt.ylabel('Rótulo Verdadeiro');plt.show()
    except FileNotFoundError:print(f"Arquivo de checkpoint '{checkpoint_path}' não encontrado.")
    except Exception as e:print(f"Ocorreu um erro na avaliação final: {e}")